In [38]:
import pandas as pd
import json
from jinja2 import Environment, FileSystemLoader
from selenium import webdriver
import os

## Load Latest Data to Dictionary (for simple code access reasons)

In [39]:
# Get Data from csv, filter for Stirling Council, strip uneeded columns and convert Changes Json from String to Dictionary
LatestData = pd.read_csv('Data Files//Latest Values.csv', converters={'Changes': json.loads}).query(f'Local_Authority == "Stirling"')[['Code','Period','Real_Value','FamilyRank','Previous_Row','Changes']]
# Get indicator info from csv and merge titles into LatestData
Info = pd.read_csv('Data Files//Indicator Information.csv')
LatestData = LatestData.merge(Info[['Code_Sortable','Title','Format_Python','AdditionalAxisDenominator_Python']], how='left', left_on=['Code'], right_on=['Code_Sortable'], suffixes = ('_latest','_info'))
def formatvalue (df) : 
    df['Real_Value_Formatted'] = df['Format_Python'].format(df['Real_Value']/df['AdditionalAxisDenominator_Python'])
    return df['Real_Value_Formatted']
LatestData['Real_Value_Formatted'] = LatestData.apply(formatvalue, axis = 1)
# Set the index of the dataframe to the code and then convert to a dictionary
LatestData = LatestData.set_index('Code').to_dict(orient="index")
# Indicator data can now been accessed as below by referencing the code as the key within the dictionary
LatestData['C&L 01']


{'Period': '2020-21',
 'Real_Value': 213.35156342884315,
 'FamilyRank': 8.0,
 'Previous_Row': '{"Real_Value": 2.422236746010734, "Real_Numerator": 2631527.689813045, "Real_Denominator": 1086404.0, "Cash_Value": 2.270794290153571, "Cash_Numerator": 2467000.0, "Cash_Denominator": 1086404.0, "ScotRank": 13.0, "ScotPct": 0.40625, "FamilyRank": 4.0, "FamilyPct": 0.5}',
 'Changes': {'ScotRank_ChangeSincePrevious': -18.0,
  'ScotPct_ChangeSincePrevious': -0.5625,
  'FamilyRank_ChangeSincePrevious': -4.0,
  'FamilyPct_ChangeSincePrevious': -0.5,
  'ScotRank_ChangeSinceFirst': -21.0,
  'ScotPct_ChangeSinceFirst': -0.65625,
  'FamilyRank_ChangeSinceFirst': -5.0,
  'FamilyPct_ChangeSinceFirst': -0.625,
  'Real_Value_ChangeSincePrevious': 210.92932668283242,
  'Real_Numerator_ChangeSincePrevious': 957472.3101869547,
  'Real_Denominator_ChangeSincePrevious': -1069582.0,
  'Cash_Value_ChangeSincePrevious': 211.08076913868962,
  'Cash_Numerator_ChangeSincePrevious': 1122000.0,
  'Cash_Denominator_Cha

In [40]:
env = Environment(loader=FileSystemLoader('Templates_Jinja2'))

In [41]:
def createGroupPage(pagename, layouttitle) :
    template = env.get_template(layouttitle)
    htmlout = template.render(LatestData = LatestData)
    file = open("Pages_FinalReport/" + pagename, "w")
    file.write(htmlout)
    file.close()

def savetopdf (relativepath) :
    chrome_options = webdriver.ChromeOptions()
    #headless breaks the pdf outputting for some reason
    chrome_options.add_argument("--headless")
    
    #settings or prefs here need altered to specify A3, landscape, no header and footer and print css backgrounds. Ideally also download path so that changes are saved directly into a folder in the repo files.
    settings = {"recentDestinations": [{"id": "Save as PDF", "origin": "local", "account": ""}], "selectedDestinationId": "Save as PDF", "version": 2}
    prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings)}
    chrome_options.add_experimental_option('prefs', prefs)
    chrome_options.add_argument('--kiosk-printing')
    
    #something here is depricated and requires replaced as per error output when used
    browser = webdriver.Chrome(r"C:\\Program Files\\Google\\Chrome\\Application\\chromedriver.exe", options=chrome_options)
    htmlfilepath = "file:///" +  os.path.abspath(relativepath)
    browser.get(htmlfilepath)
    browser.execute_script('window.print();')

In [42]:
createGroupPage("Waste.html","Layout_Waste.html")
savetopdf('./Pages_FinalReport/Waste.html')

<ipython-input-41-f87c046b25c9>:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r"C:\\Program Files\\Google\\Chrome\\Application\\chromedriver.exe", options=chrome_options)
